In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import json
import os

with open('../../Secret_file.json',"r") as f:
    ks_token = json.load(f)['Slack_API']['KS']

In [ ]:
target_url = 'https://www.kickstarter.com/projects/moaideas/steel-n-steam-hexagonal-metal-coins-for-train-games/'

In [ ]:
def Bot_Message(channel, text):
    payload={
        "token":ks_token,
        "channel":channel,
        "text":'{}'.format(text)} 
    r = requests.post("https://slack.com/api/chat.postMessage", params=payload) 

def str2num(x):
    temp = ''
    for i in x:
        if i.isdigit() or i == '.':
            temp += i
    return temp

def json_save(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f)

def json_load(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

class parse_stats_data():
    def __init__(self, url):
        self.url = url
        self.json = requests.get(url).json()['project']
        
    def state(self):
        return self.json['state']
    
    def data(self):
        return dict(self.json)

class parse_back_data():
    def __init__(self, url):
        self.url = url
        self.soup = BeautifulSoup(requests.get(url).text, 'html5lib')
        
    def data(self):
        temp = {}
        for x in self.soup.select('li.hover-group')[1:]:
            name = 'project_' + str2num(x.select_one('.money').text)
            if name not in temp:
                pass
            else:
                name += '_2ed'
            temp[name] = eval(str2num(x.select_one('.text-nowrap').text))
        return temp

In [ ]:
'''
時間阻斷器
'''
# 暫時只用小時設定
def time_start(hour):
    # 時
    while True:
        if datetime.now().hour == (hour-1):
            break
        else:
            time.sleep(3600)
            continue

    # 分
    while True:
        if datetime.now().minute == 59:
            break
        else:
            time.sleep(60)
            continue

    # 秒
    while True:
        if datetime.now().second >= 50:
            break
        else:
            time.sleep(5)
            continue

    while True:
        if datetime.now().hour == hour:
            break
        else:
            continue
time_start(15)
print('執行時間：\n', datetime.now())

In [ ]:
if __name__ == "__main__":
    if os.path.isfile('火車幣_savedata.json'):
        data = json_load('火車幣_savedata.json')
    else:
        json_save({}, '火車幣_savedata.json')
        data = json_load('火車幣_savedata.json')
    
    while True:
        try:
            target = parse_stats_data(target_url + 'stats.json?v=1')
            break
        except:
            time.sleep(1)
    start_time = time.time() #1 首次5分計時器
    check_point = 0
    while target.state() == 'live':
        try:
            temp = target.data()
            temp.update(parse_back_data(target_url).data())
            data[datetime.now().strftime('%Y-%m-%d %H:%M:%S')] = temp
            json_save(data, '火車幣_savedata.json')

            time.sleep(300-(time.time()-start_time)) #1 5分計時器
            target = parse_stats_data(target_url + 'stats.json?v=1') #程序後請求
            start_time = time.time() #1 5分計時器
            
        except:
            check_point += 1 #2 【偵錯器】
            Bot_Message('spider_status', '<@D.E.>\nGet ERROR for KS_老外火車幣 !({}/{})\nPlease check the codes or website .'.format(check_point, 5))
            if check_point == 5:
                break
            else:
            time.sleep(5)